In [1]:
#!pip install -q -U transformers accelerate peft datasets
#!pip install pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 2.9 MB/s eta 0:00:00a 0:00:01m


In [1]:
import torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
)

from peft import PeftModel

# Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [2]:
# Model name/path on Hugging Face Hub
# Choose model you want to use
model_name_or_path = 'Yuta555/Llama-2-7b-MBTI-classification' # 16-class model
#model_name_or_path = 'Yuta555/Llama-2-7b-MBTI-binary-clf-1st' # Binary model 1st dimension (E/I)
#model_name_or_path = 'Yuta555/Llama-2-7b-MBTI-binary-clf-2nd' # Binary model 2nd dimension (N/S)
#model_name_or_path = 'Yuta555/Llama-2-7b-MBTI-binary-clf-3rd' # Binary model 3rd dimension (F/T)
#model_name_or_path = 'Yuta555/Llama-2-7b-MBTI-binary-clf-4th' # Binary model 4th dimension (J/P)

# Load configration, model, and tokenizer
config = AutoConfig.from_pretrained(model_name_or_path)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path,
    config=config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_flash_attention_2=True, # speed up inference
)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, model_name_or_path)

# Merge the adapter weights with the base model which allows the model's inference to speed up
merged_model = model.merge_and_unload()

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/489 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.bin:   0%|          | 0.00/33.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Prediction Function

In [3]:
# Function to prediction using tweets
def predict(tweet: str):
    """
    Input: Tweet texts (str). It should include 50 tweets and be indexed in the format below:
        -----
        1. XXX
        2. XXX
        ...
        -----
    Return: Predicted MBTI label in the form of a four-letter alphabet (str).
    """

    inputs = tokenizer(tweet, return_tensors="pt").to('cuda')

    with torch.no_grad():
        outputs = merged_model(**inputs)
        predictions = outputs.logits.argmax(dim=-1).item()
        predictions = model.config.id2label[predictions]

    return predictions

## Sample Code for Prediction

In [ ]:
tweets = ... # Processed tweets text in the indexed format

pred_label = predict(tweets)
print(pred_label)

### Check with test data

In [6]:
# Sample code
dimension_id = None # None/0/1/2/3


from datasets import load_from_disk
from tqdm.notebook import tqdm

dataset = load_from_disk("data/processed_data_50tweets")
#############################
# Added to change to binary classification

# Extract Nth dimension from 4-dimension labels
def extract_nth_dimension(example, n):
    new_label = example['label'][n]
    return {'label': new_label}

if dimension_id is not None:
    assert dimension_id in range(0, 4), f"dimension_id must be None or between 0 and 3"
    dataset = dataset.map(lambda example: extract_nth_dimension(example, dimension_id))
#############################

test_data = dataset['test']

preds = []
for dp in tqdm(test_data):
    pred_label = predict(dp['text'])
    preds.append(pred_label)

  0%|          | 0/868 [00:00<?, ?it/s]

In [7]:
# Save results
import json

results = {
    'text': dataset['test']['text'],
    'references': dataset['test']['label'],
    'predictions': preds
}

with open("test_results.json", "w") as f:
    json.dump(results, f)

In [8]:
# Evaluation
refs = dataset['test']['label']
acc = sum([ref == pred for ref, pred in zip(refs, preds)])
accuracy = acc / len(preds)
print(accuracy)

0.8168202764976958
